In [4]:
import cv2                            # Importing Opencv library for image processing and computer vision
import numpy as np                    # Importing numpy library 
import matplotlib.pyplot as plt       # Importing matplotlib library to plot the images directly in notebook
from skimage.metrics import structural_similarity as ssim # Importing ssim calculating modules from skimage library
from tkinter import * # Importing tkinter library to build GUI
from PIL import Image as PIL_Image
from PIL import ImageTk
import tkinter.filedialog as tkFileDialog
from tkinter import messagebox
import time



In [13]:
path = r
%store path

# A sample path:
# path = r'Dataset\500_dataset\500_s1.jpg'

print('Path of input image: ', path)

NameError: name 'r' is not defined

In [14]:
# Reading the image
test_img = cv2.imread(path)

NameError: name 'path' is not defined

In [ ]:
# Pre- processing

# Resizing the image
test_img = cv2.resize(test_img, (1167, 519))

# Guassian Blur
blur_test_img = cv2.GaussianBlur(test_img, (5,5), 0)

# Grayscale conversion
gray_test_image = cv2.cvtColor(blur_test_img, cv2.COLOR_BGR2GRAY)

def preprocessing():
    # Showing original currency note
    plt.imshow(gray_test_image, 'gray')
    plt.title('Input image after pre- processing')
    plt.show()
    
    # Updating the progress
    progress['value']=5
    ProgressWin.update_idletasks()

In [ ]:
#  Calculating SSIM of the two images sent as parameters

def calculateSSIM(template_img, query_img):
    min_w = min(template_img.shape[1], query_img.shape[1])
    min_h = min(template_img.shape[0], query_img.shape[0])
    
    # Resizing the two images so that both have same dimensions
    img1 = cv2.resize(template_img, (min_w, min_h))
    img2 = cv2.resize(query_img, (min_w, min_h))
    
    # Conversion to gray- scale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Plotting the images
    plt.subplot(1, 2, 1)
    plt.imshow(img1, 'gray')

    plt.subplot(1, 2, 2)
    plt.imshow(img2, 'gray')

    plt.show()
    
    # Find the SSIM score and return
    score = ssim(img1, img2)
    return score

In [ ]:
# Feature detection using ORB

def computeORB(template_img, query_img):
    # ===================== Creating orb object ==============================

    nfeatures=700;
    scaleFactor=1.2;
    nlevels=8;
    edgeThreshold=15; # Changed default (31);

    # Initialize the ORB detector algorithm 
    orb = cv2.ORB_create(
        nfeatures,
        scaleFactor,
        nlevels,
        edgeThreshold)
    # =========================================================================
    
    # Find the keypoints and descriptors with ORB
    # This will find the keypoints of each of the image and then find the descriptors corresponding to each keypoint.
    
    kpts1, descs1 = orb.detectAndCompute(template_img,None)
    kpts2, descs2 = orb.detectAndCompute(query_img,None)
    
    # ====================================== Brute Force Matching ==============================
    # Starting a brute force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Finding matches between the 2 descrptor sets
    matches = bf.match(descs1, descs2)

    # sort the matches in the order of their distance
    # Lower the distance, better the matching
    dmatches = sorted(matches, key = lambda x:x.distance)

    # ===========================================================================================
    
    # ======================================= Image homography ================================
    ## extract the matched keypoints
    src_pts  = np.float32([kpts1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
    dst_pts  = np.float32([kpts2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)

    ## find homography matrix and do perspective transform
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    h,w = template_img.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    
    if M is not None:
        dst = cv2.perspectiveTransform(pts,M)
    else:
        dst = None

    # This finds the template region in the test currency note
    # ====================================================================================
    
    # Returning necessary data
    return dst, dst_pts, kpts1, kpts2, dmatches

In [ ]:
# Values for specifying search area of features 1 to 7

search_area_list = [[200,300,200,370],
                    [1050,1500,300,450],
                    [100,450,20,120],
                    [690,1050,20,120],
                    [820,1050,350,430],
                    [700,810,330,430],
                    [400,650,0,100]]

# Values of max_area and min_area for each feature for features 1 to 7

feature_area_limits_list = [[12000,17000],
                            [10000,18000],
                            [20000,30000],
                            [24000,36000],
                            [15000,25000],
                            [7000,13000],
                            [11000,18000]]

score_set_list = []               # Stores the ssim score set of each feature
best_extracted_img_list = []      # Stores the extracted image with highest SSIM score for each feature
avg_ssim_list = []                # Stores the avg ssim value for each feature
NUM_OF_FEATURES = 7               # Number of features

In [ ]:
# Algo- 1: Verification of features 1 to 7

def testFeature_1_2_7():
    i = 0
    j = 0
    NUMBER_OF_TEMPLATES = 6 
    global score_set_list                # Stores the ssim score set of each feature
    global best_extracted_img_list       # Stores the extracted image with highest SSIM score for each feature
    global avg_ssim_list                 # Stores the avg ssim value for each feature

    
    #Progress bar
    global myProgress
    myProgress =progress['value']

    
    # Iterating for each feature
    for j in range(NUM_OF_FEATURES):
        print('ANALYSIS OF FEATURE ' + str(j+1))

        score_set = []           # SSIM scores for each teamplate of current feature will be stored here
        max_score = -1           # Stores max SSIM score
        max_score_img = None     # Stores extraced image with max SSIM score for the current feature

        # Performing feature detection, extraction and comparison for each template stored in dataset 
        for i in range(NUMBER_OF_TEMPLATES):
            print('---> Template ' + str(i+1) + ' :')

            # Current template 
            template_path = r'Dataset\500_Features Dataset\Feature ' + str(j+1) + '\\' + str(i+1) + '.jpg'

            template_img = cv2.imread(template_path)

            template_img_blur = cv2.GaussianBlur(template_img, (5,5), 0)
            template_img_gray = cv2.cvtColor(template_img_blur, cv2.COLOR_BGR2GRAY)

            test_img_mask = gray_test_image.copy()

            # Creating a mask to search the current template.
            search_area = search_area_list[j]

            test_img_mask[:, :search_area[0]] = 0
            test_img_mask[:, search_area[1]:] = 0
            test_img_mask[:search_area[2], :] = 0
            test_img_mask[search_area[3]:, :] = 0

            # Feature detection using ORB 
            dst, dst_pts, kpts1, kpts2, dmatches = computeORB(template_img_gray, test_img_mask)

            # Error handling
            if dst is None:
                print('An Error occurred - Homography matrix is of NoneType')
                continue

            query_img = test_img.copy()

            # drawing polygon around the region where the current template has been detected on the test currency note -- the blue polygon
            res_img1 = cv2.polylines(query_img, [np.int32(dst)], True, (0,0,255), 1, cv2.LINE_AA)

            # draw match lines between the matched descriptors
            res_img2 = cv2.drawMatches(template_img, kpts1, res_img1, kpts2, dmatches[:20],None,flags=2)

            # Find the details of a bounding rectangle that bounds the above polygon --- green rectangle
            (x, y, w, h) = cv2.boundingRect(dst) # This gives us details about the rectangle that bounds this contour  

            # Checking if the area of the detected region is within the min and max area allowed for current feature 
            min_area = feature_area_limits_list[j][0]
            max_area = feature_area_limits_list[j][1]

            feature_area = w*h
            if feature_area < min_area or feature_area > max_area:
                (x, y, w, h) = cv2.boundingRect(dst_pts) # naya rectangle banaya

                feature_area = w*h
                if feature_area < min_area or feature_area > max_area: # If even area of 2nd rect is outside limits, then Discard 
                    # If even area of 2nd rect is outside limits, then Discard current template
                    print('Template Discarded- Area of extracted feature is outside permitted range!')
                    continue

            # Draw the rectangle
            cv2.rectangle(res_img1, (x,y), (x+w, y+h), (0,255,0), 3)

            # Plotting images
            plt.rcParams["figure.figsize"] = (16, 16)
            plt.subplot(1, 2, 1)
            plt.imshow(res_img2, 'gray')

            plt.subplot(1, 2, 2)
            plt.imshow(res_img1, 'gray')
            plt.show()

            # SSIM
            # Crop out the region inside the green rectangle (matched region)
            crop_img = blur_test_img[y:y+h, x:x+w]

            plt.rcParams["figure.figsize"] = (5, 5)
            score = calculateSSIM(template_img_blur, crop_img)

            score_set.append(score)
            print('SSIM score: ', score, '\n')

            # Keeping details about extracted region with highest SSIM score
            if score > max_score:
                max_score = score
                max_score_img = crop_img
            
            #Progress bar- Updating the progess
            myProgress = myProgress + (75.0/(NUM_OF_FEATURES*NUMBER_OF_TEMPLATES))
            progress['value'] = myProgress 
            ProgressWin.update_idletasks()
            
        # Storing necessary data
        score_set_list.append(score_set)
        print('SSIM score set of Feature ' + str(j+1) + ': ', score_set, '\n')

        avg_ssim_list.append(sum(score_set)/len(score_set))
        print('Average SSIM of Feature ' + str(j+1) + ': ',sum(score_set)/len(score_set),'\n')
        
        if len(score_set) != 0:
            avg_ssim_list.append(sum(score_set)/len(score_set))
            print('Average SSIM of Feature ' + str(j+1) + ': ',sum(score_set)/len(score_set),'\n')
        else:
            print('No SSIM scores were found for this feature!')
            avg_ssim_list.append(0.0)
            print('Average SSIM of Feature ' + str(j+1) + ': 0','\n')

        best_extracted_img_list.append([max_score_img, max_score])

    # Printing all details for features 1- 7
    print('Final Score- set list:','\n')

    for x in range(len(score_set_list)):
        print('Feature',x+1,':',score_set_list[x])
    print('\n')

    print('Final Average SSIM list for each feature:','\n')

    for x in range(len(avg_ssim_list)):
        print('Feature',x+1,':',avg_ssim_list[x])


In [ ]:
left_BL_result = []
right_BL_result = []
result_list = []
number_panel_result = []

In [ ]:
# Function to count number of bleed lines in left side- Feature 8
# Algo 2

def testFeature_8():
    plt.rcParams["figure.figsize"] = (5, 5)

    # Check Feature 8- Left bleed lines
    print('\nANALYSIS OF FEATURE 8 : LEFT BLEED LINES\n')

    # Cropping the region in which left bleed lines are present- Feature extraction
    crop = test_img[120:240, 12:35]

    img = crop.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

    plt.imshow(thresh, 'gray')
    plt.show()

    whitePixelValue = 255      # White pixel   
    blackPixelValue = 0        # Black pixel

    width = thresh.shape[1]    # width of thresholded image

    # Result will be stored here
    result = []                # will contain the number of black regions in each column (if the colums is non- erroneos)
    num_of_cols = 0            # will contain the number of non- erroneos columns

    # Non erroneous columns are those columns which contains less noise.

    print('Number of black regions found in each column: ')

    # iteration over each column in the cropped image
    for j in range(width):
        col =thresh[:, j:j+1]     # Extracting each column of thresholded image
        count = 0                 # Counter to count number of black regions in each extracted column

        # Iterating over each row (or pixel) in the current columm
        for i in range(len(col)-1):

            # Taking two consecurive pixels and storing their intensity value
            pixel1_value = col[i][0]
            pixel2_value = col[i+1][0]

            #----------------------------------------------
            # This part modifies any error pixels, if present.
            # Actually in a binary threshold, all pixels should be either white or black.
            # If due to some error pixels other than white or black are present, then the pixel is taken as white pixel

            if pixel1_value != 0 and pixel1_value != 255:
                pixel1_value = 255
            if pixel2_value != 0 and pixel2_value != 255:
                pixel2_value = 255

            #-------------------------------------------------


            # If current pixel is white and next pixel is black, then increment the counter.
            # This shows that a new black region has been discovered.
            if pixel1_value == whitePixelValue and pixel2_value == blackPixelValue:
                count += 1

        # If the counter is less than 10, it is a valid column. (less noise is present)
        if count > 0 and count < 10:
            print(count)
            result.append(count)
            num_of_cols += 1
        else:
            # discard the count if it is too great e.g. count> 10 (Erroneous Column)
            # This removes/ drops those columns which contain too much noise
            print(count, 'Erroneous -> discarded')

    # Printing necessary details
    print('\nNumber of columns examined: ', width)
    print('Number of non- erroneous columns found: ', num_of_cols)

    if num_of_cols != 0:
        average_count = sum(result)/num_of_cols
    else:
        average_count = -1
        print('Error occured- Division by 0')

    print('\nAverage number of black regions is: ', average_count)

    # Storing the thresholded image and average number of bleed lines detected 
    global left_BL_result
    left_BL_result = [thresh, average_count]
    
    # Updating progess in progress bar
    global myProgress
    progress['value']=80
    ProgressWin.update_idletasks()

In [ ]:
# Function to count number of bleed lines in right side- Feature 9

def testFeature_9():
    plt.rcParams["figure.figsize"] = (5, 5)

    # Check Feature 9- Right bleed lines
    print('\nANALYSIS OF FEATURE 9 : RIGHT BLEED LINES\n')

    # Cropping the region in which left bleed lines are present- Feature extraction
    crop = test_img[120:260, 1135:1155]

    img = crop.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Thresholding the image
    _, thresh = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

    plt.imshow(thresh, 'gray')
    plt.show()

    whitePixelValue = 255      # White pixel   
    blackPixelValue = 0        # Black pixel

    width = thresh.shape[1]    # width of thresholded image

    # Result will be stored here
    result = []                # will contain the number of black regions in each column (if the colums is non- erroneos)
    num_of_cols = 0            # will contain the number of non- erroneos columns

    # Non erroneous columns are those columns which contains less noise.

    print('Number of black regions found in each column: ')

    # Iteration over each column in the cropped image
    for j in range(width):
        col =thresh[:, j:j+1]     # Extracting each column of thresholded image
        count = 0                 # Counter to count number of black regions in each extracted column

        # Iterating over each row (or pixel) in the current columm
        for i in range(len(col)-1):

            # Taking two consecurive pixels and storing their intensity value
            pixel1_value = col[i][0]
            pixel2_value = col[i+1][0]

            #----------------------------------------------
            # This part modifies any error pixels, if present.
            # Actually in a binary threshold, all pixels should be either white or black.
            # If due to some error pixels other than white or black are present, then the pixel is taken as white pixel

            if pixel1_value != 0 and pixel1_value != 255:
                pixel1_value = 255
            if pixel2_value != 0 and pixel2_value != 255:
                pixel2_value = 255

            #-------------------------------------------------

            # If current pixel is white and next pixel is black, then increment the counter.
            # This shows that a new black region has been discovered.
            if pixel1_value == whitePixelValue and pixel2_value == blackPixelValue:
                count += 1

        # If the counter is less than 10, it is a valid column. (less noise is present)
        if count > 0 and count < 10:
            print(count)
            result.append(count)
            num_of_cols += 1
        else:
            # discard the count if it is too great e.g. count> 10 (Erroneous Column)
            # This removes/ drops those columns which contain too much noise
            print(count, 'Erroneous -> discarded')

    # Printing necessary details
    print('\nNumber of columns examined: ', width)
    print('Number of non- erroneous columns found: ', num_of_cols)

    if num_of_cols != 0:
        average_count = sum(result)/num_of_cols
    else:
        average_count = -1
        print('Error occured- Division by 0')

    print('\nAverage number of black regions is: ', average_count)

    # Storing the thresholded image and average number of bleed lines detected 
    global right_BL_result
    right_BL_result = [thresh, average_count]
    global myProgress
    
    # Updating progess in progress bar
    progress['value']=85
    ProgressWin.update_idletasks()

In [ ]:
# Cropping out the number panel.

# Cropping out the number panel. - FEATURE EXTRACTION

crop = gray_test_image[410:500, 700:1080]            # This is the cropped gray image
crop_bgr = test_img[410:500, 700:1080]               # This is the cropped BGR image



In [ ]:
# Algo 3

def testFeature_10():
    plt.imshow(crop_bgr)
    plt.show()

    plt.rcParams["figure.figsize"] = (7, 7)

    print('\nANALYSIS OF FEATURE 10 : NUMBER PANEL \n')

    test_passed = False        # If true, then the test is successful
    res_img_list = []          # List of images of successful cases
    count = 0                  # Stores number of cases whihc are successful
    i = 0

    # THRESHOLDING at multiple values
    # Start from 95 as threshold value, increase the threshold value by 5 every time and check if 9 characters are detected in the thresholded image of number panel
    # If 9 characters are detected in at least one of the cases, the currency number panel is verified.
    # If more than 1 cases pass, the best image will be choosen from the successful cases.
    for thresh_value in range(95, 155, 5):
        # Thresholding at current value
        _, thresh = cv2.threshold(crop, thresh_value, 255, cv2.THRESH_BINARY)

        print('---> Threshold ' + str(i+1) + ' with Threshold value ' + str(thresh_value) + ' :')
        i += 1

        copy = crop_bgr.copy()

        # Finding all the contours in the image of the number panel- CONTOUR DETECTION 
        img = cv2.bitwise_and(crop, crop, mask=thresh)
        contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        h_img, w_img = img.shape[:2]

        # cv2.drawContours(copy, contours, -1, (0, 0, 255), 1)

        # Storing the details of all the BOUNDING RECTANGLES for each contour
        bounding_rect_list = []    

        for contour in contours:
            [x, y, w, h] = cv2.boundingRect(contour)

            if x != 0:
                bounding_rect_list.append([x,y,w,h])

        # Sorting the list of rectangles
        # Rectangles will get sorted according to the x coordinate of the top left corner
        bounding_rect_list.sort()

        # ELIMINATION OF ERRONEOUS RECTANGLES
        # Min area is taken as 150
        min_area = 150
        res_list = []

        # Storing all rectangles having area greater than the min_area in a separate list
        for i in range(0, len(bounding_rect_list)):
            if i>= len(bounding_rect_list):
                break
            if bounding_rect_list[i][2]*bounding_rect_list[i][3] > min_area:
                res_list.append(bounding_rect_list[i])

        # Eliminating the rectangles that are present within a bigger rectangle
        i = 0
        while i<len(res_list):
            [x,y,w,h] = res_list[i]
            j = i+1
            while j<len(res_list):
                [x0,y0,w0,h0] = res_list[j]

                if (x+w) >= x0+w0:
                    res_list.pop(j)
                else:
                    break
            i+= 1

        # Eliminating unnecessary rectangles
        i = 0
        while i<len(res_list):
            [x,y,w,h] = res_list[i]

            if (h_img-(y+h)) > 40:     #  Eliminating the rectangles whose lower edge is further away from lower edge of the image 
                res_list.pop(i)
            elif h<17:
                res_list.pop(i)        # Eliminating the rectangles whose height is less than 17 pixels    
            else:
                i += 1

        for rect in res_list:          # Drawing the remaining rectangles
            [x,y,w,h] = rect
            cv2.rectangle(copy, (x, y), (x + w, y + h), (0, 0, 255), 1)        

        # COUNTING REMAINING RECTANGLES
        # result of each image
        if len(res_list) == 9:         # If number of rectangles detected is greater than 9, test passed
            test_passed = True
            res_img_list.append(copy)
            count += 1
            print('Test Successful: 9 letters found!')
        else:
            print('Unsuccessful!')


        # If three consecutive cases pass the test, then break 
        if count == 3:
            break

    # Choosing the BEST IMAGE to be displayed   
    # Even if a single case passes the test, then the currency number panel is verified.
    # Selecting the best image to display
    if count == 0:                    # If none of the cases passes the test
        best_img = crop_bgr
    elif count == 1:                  # If 1 case passes the test, then the image used in 1st case is selected as the best image
        best_img = res_img_list[0]
    elif count == 2:                  # If 2 cases pass the test, then the image used in 2nd case is selected as best image
        best_img = res_img_list[1]
    else:                             # If >= 3 cases pass the test, then the image used in 3rd case is selected as best image
        best_img = res_img_list[2]
    
    
    # Displaying final result

    if(test_passed):
        print('Test Passed!- 9 characters were detected in the serial number panel.')
        plt.imshow(best_img)
        plt.show()
    else:
        print('Test Failed!- 9 characters were NOT detected in the serial number panel.')

    # Storing the thresholded image and the result
    global number_panel_result
    number_panel_result = [best_img, test_passed]
    
    # Updating progress in progress bar
    global myProgress
    progress['value']=90
    ProgressWin.update_idletasks()

In [ ]:
# Result analysis

def testResult():
    plt.rcParams["figure.figsize"] = (3, 3)

    print('\n\nRESULT ANALYSIS\n')

    # Stores the min allowed SSIM score for each feature
    min_ssim_score_list = [0.4, 0.4, 0.5, 0.4, 0.5, 0.45, 0.5]

    global result_list
    result_list = []               # To store details of each feature
    successful_features_count = 0  # To store number of features which passed the test

    # Feature 1 to 7: Results
    for i in range(NUM_OF_FEATURES):
        avg_score = avg_ssim_list[i]
        img, max_score = best_extracted_img_list[i]
        status = False
        min_allowed_score = min_ssim_score_list[i]

        # A feature passes the test if its avg SSIM score is greater than a min. decided value 
        # or if its max SSIM score is greater than 0.8
        if avg_score >= min_allowed_score or max_score >= 0.79:
            status = True
            successful_features_count += 1
            print('Feature ' + str(i+1) + ': Successful')
        else:
            status = False
            print('Feature ' + str(i+1) + ': Unsuccessful')
        
        if img is None:
            img = cv2.imread('Image_not_found.jpg')
        
        result_list.append([img, avg_score, max_score, status])

        
    # Feature 8: Left Bleed lines

    img, line_count = left_BL_result[:]

    # The feature passes the test if number of bleed lines is close to 7 (6.7 - 7.6)
    if line_count >= 4.7 and line_count <= 5.6:
        status = True
        successful_features_count += 1
        print('Feature 8: Successful- 5 bleed lines found in left part of currency note')
    else:
        status = False
        print('Feature 8: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, line_count, status])

    
    # Feature 9: Right Bleed lines

    img, line_count = right_BL_result[:]

    # The feature passes the test if number of bleed lines is close to 7 (6.7 - 7.6)
    if line_count >= 4.7 and line_count <= 5.6:
        status = True
        successful_features_count += 1
        print('Feature 9: Successful- 5 bleed lines found in right part of currency note')
    else:
        status = False
        print('Feature 9: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, line_count, status])

    
    # Feature 10: Currency Number Panel

    img, status = number_panel_result[:]

    # The feature passes the test if 9 characters are detected in the number panel
    if status:
        successful_features_count += 1
        print('Feature 10: Successful- 9 characters found in number panel of currency note')
    else:
        print('Feature 10: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, status])

    # printing FINAL RESULT

    print('\nResult Summary:')
    print(str(successful_features_count) + ' out of 10 features are VERIFIED!')
    
    # Updating progress bar
    global myProgress
    progress['value']=97
    ProgressWin.update_idletasks()

In [8]:
root = Tk()
root.geometry("800x1000")
root.title("Fake Currency Testing")

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

def select_image():
    # grab a reference to the image canvas and the path variable
    global canvas
    global path
    
    canvas.delete("all")
    # open a file chooser dialog and allow the user to select an input
    # image
    path = tkFileDialog.askopenfilename()

    # ensure a file path was selected
    if len(path) > 0 and path[-4:] == '.jpg':
        # load the image from disk
        image = cv2.imread(path)
        original_image = image.copy()

        #  represents images in BGR order; however PIL represents
        # images in RGB order, so we need to swap the channels
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (675, 300))

        # convert the images to PIL format...
        image = Image.fromarray(image)

        # ...and then to ImageTk format
        image = ImageTk.PhotoImage(image)

        canvas.image = image
        canvas.create_image(0, 0, anchor=NW, image=image) 
    else:
        messagebox.showinfo("Warning","Please choose an image! (It should be a .jpg file)")



def submit():
    global option
    global path
    global selectedImage
    if len(path) <= 0:
        messagebox.showinfo("Warning","Please choose an image!")
    else:
        print(path)
        print(option)
        selectedImage = True
        messagebox.showinfo("Info","Image sent for processing!")
        root.destroy()
        
        
        def display_output():
            sub_frame1 = Frame(master_frame, bg='black', pady=5)
            sub_frame2 = Frame(master_frame, bg='brown', pady=5, padx = 5)
            sub_frame3 = Frame(master_frame, pady=5, padx = 5)
            sub_frame4 = Frame(master_frame, pady=5, padx = 5)
            
            sub_frame1.grid(row = 1, column = 1, padx = 5, pady = 5)
            sub_frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
            sub_frame3.grid(row = 3, column = 1, padx = 5, pady = 5)
            sub_frame4.grid(row = 4, column = 1, padx = 5, pady = 5)
            
            title = Label(master=sub_frame1, text="FAKE CURRENCY DETECTION SYSTEM", fg = 'dark blue', font = "Verdana 28 bold")
            title.pack()
            
            canvaas_input = Canvas(master=sub_frame2, width = 675, height = 300)  
            canvaas_input.pack()
            
            if len(path) > 0 and path[-4:] == '.jpg':
                # load the image from disk
                image = cv2.imread(path)
                original_image = image.copy()

                #  represents images in BGR order; however PIL represents
                # images in RGB order, so we need to swap the channels
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (675, 300))

                # convert the images to PIL format...
                image = PIL_Image.fromarray(image)

                # ...and then to ImageTk format
                image = ImageTk.PhotoImage(image)

                canvaas_input.image = image
                canvaas_input.create_image(0, 0, anchor=NW, image=image) 
    
            pass_count = 0
    
            # Displaying analysis of each feature in sub_frame4
            # Looping over result_list and displaying the data for each feature
            for i in range(4):
                for j in range(3):
                    feature_num = 3*i+j      # This can vary from 0 to 9
            
                    if feature_num < 10:     # There are 10 features
                        sub_frame4.grid_rowconfigure(i, weight=1)
                        sub_frame4.grid_columnconfigure(j, weight=1)
                
                        # Creating a frame to display each image
                        feature_frame = Frame(master = sub_frame4, relief = RAISED, borderwidth = 1, bg='light blue')
                        feature_frame.grid(row = i, column = j, padx = 20, pady = 20, sticky="nsew")
                
                        # Creating frames inside the feature_frame to display the details of a feature
                        frame1 = Frame(feature_frame, padx=3, pady=3)
                        frame2 = Frame(feature_frame, bg='brown', pady=5, padx = 5)
                        frame3 = Frame(feature_frame)
                        frame4 = Frame(feature_frame)
                        frame5 = Frame(feature_frame)
                
                        # Assigning a grid layout
                        frame1.grid(row = 1, column = 1, padx = 5, pady = 5, ipadx = 100)
                        frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
                        frame3.grid(row = 3, column = 1, padx = 5, pady = 5)
                        frame4.grid(row = 4, column = 1, padx = 5, pady = 5)
                        frame5.grid(row = 5, column = 1, padx = 5, pady = 5)
                
                        # Displaying the feature number through a label in frame1 ------------
                        label1 = Label(master = frame1, text = f"Feature {feature_num +1}", fg = 'black', font = "Verdana 12 bold")
                        label1.pack()

                        # Creating a canvas to display the image in frame2 ------------
                        canvaas = Canvas(master=frame2, width = 200, height = 200)  
                        canvaas.pack()

                        image = result_list[feature_num][0].copy()

                        h, w = image.shape[:2]
                        aspect_ratio = w/h

                        resize_height = 0
                        resize_width = 0
                        img_x = 0
                        img_y = 0
                
                        if h > w:
                           resize_height = 200
                           resize_width = aspect_ratio * resize_height
                           img_x = (200 - resize_width)/2
                        elif h < w:
                           resize_width = 200
                           resize_height = resize_width / aspect_ratio
                           img_y = (200 - resize_height)/2
                        else:
                            resize_height = 200
                            resize_width = 200
                
                        resize_height = int(resize_height)
                        resize_width = int(resize_width)
                        img_x = int(img_x)
                        img_y = int(img_y)

                        # Resizing the image while maintaining the aspect ratio
                        image = cv2.resize(image, (resize_width, resize_height))

                        # convert the images to PIL format...
                        image = PIL_Image.fromarray(image)

                        # ...and then to ImageTk format
                        image = ImageTk.PhotoImage(image)

                        # Show the image in canvas
                        canvaas.image = image
                        canvaas.create_image(img_x, img_y, anchor=NW, image=image) 
                
                        # 2nd label in frame3 ------------
                        if feature_num < 7:
                            avg_score = result_list[feature_num][1]
                            avg_score = "{:.3f}".format(avg_score)
                            text2 = "Avg. SSIM Score: " + avg_score
                        elif feature_num < 9:
                            line_count = result_list[feature_num][1]
                            line_count = "{:.3f}".format(line_count)
                            text2 = "Avg. Number of lines: " + line_count
                        else:
                            status = result_list[feature_num][1]
                        if status == True:
                            text2 = "9 characters detected!"
                        else:
                            text2 = "Less than 9 characters detected!"
                            label2 = Label(master = frame3, text = text2, fg = 'dark blue', font = "Verdana 11", bg='light blue')
                            label2.pack()
                
                        # 3rd label in frame4 ------------
                        if feature_num < 7:
                            max_score = result_list[feature_num][2]
                            max_score = "{:.3f}".format(max_score)
                            text3 = "Max. SSIM Score: " + max_score
                        elif feature_num < 9:
                            text3 = ""
                        else:
                            text3 = ""
                            label3 = Label(master = frame4, text = text3, fg = 'dark blue', font = "Verdana 11", bg='light blue')
                            label3.pack()
                
                        # 4th label in frame5 ------------
                        if feature_num < 7:
                            status = result_list[feature_num][3]
                        elif feature_num < 9:
                            status = result_list[feature_num][2]
                        else:
                            status = result_list[feature_num][1]
                
                        if status == True:
                            pass_count += 1
                            label4 = Label(master = frame5, text = "Status: PASS!", fg = 'green', font = "Verdana 11 bold", bg='light blue')
                            label4.pack()
                        else:
                            label4 = Label(master = frame5, text = "Status: FAIL!", fg = 'red', font = "Verdana 11 bold", bg='light blue')
                            label4.pack()
    
        # Result label in sub_frame3
        result = Label(master=sub_frame3, text= f"RESULT: {pass_count} / 10 features PASSED!", fg = 'green', font = "Verdana 24 bold")
        result.pack()
            
    def scrollbar_function(event):
        canvas.configure(scrollregion=canvas.bbox("all"),width=1050,height=550)
        
            
        
        new_win = Tk()
        new_win.title('Fake Currency Detection - Result Analysis')
        new_win.geometry("800x1000")
        
       
    
        # Main Function
        # Declaring root window and specifying its attributes
        
        root.resizable(False, False)
        main_frame=Frame(root,relief=GROOVE, bd=1)
        main_frame.place(x=10,y=10) # Placing the frame at (10, 10)
        
        main_frame=Frame(root,relief=GROOVE, bd=1)
        main_frame.place(x=10,y=10) # Placing the frame at (10, 10)

        # Creating a canvas inside main_frame
        canvas=Canvas(main_frame)
        master_frame=Frame(canvas)  # Creating master_frame inside the canvas

        # Inserting  and configuringscrollbar widget 
        myscrollbar=Scrollbar(main_frame,orient="vertical",command=canvas.yview)
        canvas.configure(yscrollcommand=myscrollbar.set)
        myscrollbar.pack(side="right",fill="y")
        canvas.pack(side="left")
        canvas.create_window((0,0),window=master_frame,anchor='nw')
        master_frame.bind("<Configure>",scrollbar_function)

        # Displaying output
        display_output()

        # Open the root window and loop
        root.mainloop()
     
        
def exit_window():
    root.destroy()        




path = ""
option = -1

top_frame = Frame(root, bg='black', width=1090, height=50, pady=3)
frame1 = Frame(root, bg='cyan', width=1090, height=80, padx=3, pady=3)
frame2 = Frame(root, bg='brown', width=1090, height=400, pady=5, padx = 5)
frame3 = Frame(root, width=1090, height=50, pady=3)
frame4 = Frame(root, width=1090, height=50, pady=3)
frame5 = Frame(root, bg='white', width=1090, height=20, pady=3)


top_frame.grid(row = 1, column = 1, padx = 5, pady = 5)
frame1.grid(row = 2, column = 1, padx = 5, pady = 5)
frame2.grid(row = 3, column = 1, padx = 5, pady = 5)
frame3.grid(row = 4, column = 1, padx = 5, pady = 5)
frame4.grid(row = 5, column = 1, padx = 5, pady = 5)
frame5.grid(row = 6, column = 1, padx = 5, pady = 5)

title = Label(master=top_frame, text="FAKE CURRENCY DETECTION SYSTEM", fg = 'dark blue', font = "Verdana 28 bold")
title.pack() #

text1 = Label(master=frame1, text="This is a fake currency detection sytem. Select the currency type, browse your image file and get started!", fg = 'blue', font = "Verdana 10")
text1.pack() 

canvas = Canvas(master=frame2, width = 675, height = 300)  
canvas.pack()  

btn = Button(master = frame4, text="Select an image", command=select_image , font = "Verdana 15 bold", fg='blue')
btn.pack(side = 'left', padx=10, pady=10)  

btn = Button(master = frame4, text="Submit", command = submit ,  font = "Verdana 15 bold", fg='green')
btn.pack(side = 'left', padx=10, pady=10)

#command=submit,

btn = Button(master = frame4, text="Exit", command = exit_window ,font = "Verdana 15 bold", fg='red')
btn.pack(side = 'left', padx=10, pady=10)    #command = exit_window


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\PRANETRA\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\PRANETRA\AppData\Local\Temp\ipykernel_28020\2741185773.py", line 30, in select_image
    image = Image.fromarray(image)
            ^^^^^^^^^^^^^^^
AttributeError: type object 'Image' has no attribute 'fromarray'


D:/Fake-Currency-Detection-System/Project_files/Fake Notes/500/500_f2.jpg
-1


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\PRANETRA\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\PRANETRA\AppData\Local\Temp\ipykernel_28020\2741185773.py", line 217, in submit
    result = Label(master=sub_frame3, text= f"RESULT: {pass_count} / 10 features PASSED!", fg = 'green', font = "Verdana 24 bold")
                          ^^^^^^^^^^
NameError: name 'sub_frame3' is not defined
